In [1]:
import numpy as np
import ee
import folium
import pandas as pd
from scipy.interpolate import griddata

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.preprocessing import StandardScaler

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel

from matplotlib import pyplot as plt
import seaborn as sns

In [8]:
import tqdm

In [2]:
#!earthengine authenticate
ee.Initialize()

In [3]:
def loadBathymetry(filePath,delimeter = "\t",usecols = (2,3,5),startLine = 7):

    f = open(filePath)
    textList = f.readlines()[startLine:]

    outF = open("bathyTemp.txt","w")
    for line in textList:
        line = line.replace(",",".")
        outF.write(line)    
    outF.close()

    bathy = np.loadtxt("bathyTemp.txt",delimiter = delimeter,usecols = usecols)    
    
    return bathy

In [21]:
def getWaterlineCoordinates(wm,ROI,scale = 10,level = 3):

    water_poly = wmt.reduceToVectors(geometry = ROI,scale = scale,eightConnected = False)
    water_poly = water_poly.filter(ee.Filter.eq('label',1))
    coords = water_poly.geometry().coordinates().getInfo()

    waterline = []
    if level == 3:
        for i in range(len(coords)):
            for j in range(len(coords[i])):
                for k in range(len(coords[i][j])):
                    waterline.append(coords[i][j][k])
    
    elif level == 2:
        for i in range(len(coords)):
            for j in range(len(coords[i])):                
                waterline.append(coords[i][j])

    waterline = np.array(waterline)
    waterline = np.unique(waterline,axis = 0)
    depth = np.repeat(0,len(waterline))
    depth = np.column_stack((waterline,depth))
    
    return depth

In [5]:
def getLatLonPixels(mask,ROI,scale = 10):
    
    t = tqdm.tqdm(total = 4)
    t.update(n=0)
    t.set_description(desc = "Creating LonLat image", refresh=True)
    
    lonlat = mask.pixelLonLat().multiply(mask)
    
    t.update(n=1)
    t.set_description(desc = "Creating dictionary", refresh=True)
    
    dic = lonlat.reduceRegion(reducer = ee.Reducer.toList(),geometry = ROI,scale = scale).getInfo()
    
    lat = dic['latitude']
    lon = dic['longitude']
    
    t.update(n=2)
    t.set_description(desc = "Creating array | Deleting duplicates", refresh=True)
    
    coords = np.array([lon,lat]).T
    coords = np.unique(coords,axis = 0)
    coords = coords[:len(coords)-1]
    
    t.update(n=3)
    t.set_description(desc = "Done", refresh=True)
    t.close()
    
    return coords

In [6]:
xmin = -8.2260339
ymin = 41.8596283
xmax = -8.0632989
ymax = 41.9309290

centerx = np.array([xmin,xmax]).mean()
centery = np.array([ymin,ymax]).mean()

ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

In [9]:
bathy = loadBathymetry("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/bathymetric_data_alto_lindoso")

wmt = ee.Image('users/dmlmont/TFM/WMt_Alto_Lindoso')
wmp = ee.Image('users/dmlmont/TFM/WMp_Alto_Lindoso')

coords_wmp = getLatLonPixels(wmp,ROI)
coords_wmt = getLatLonPixels(wmt,ROI)

wl = getWaterlineCoordinates(wmt,ROI)

np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Bat_wl_Alto_Lindoso.csv", np.vstack((wl,bathy)), delimiter=",")
np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Coords_wmp_Alto_Lindoso.csv",coords_wmp, delimiter=",")
np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Coords_wmt_Alto_Lindoso.csv",coords_wmt, delimiter=",")

Done: : 6it [00:04,  1.24it/s]                                                                                         
Done: : 6it [00:06,  1.02s/it]                                                                                         


In [10]:
xmin = -0.3245736
ymin = 42.6798840
xmax = -0.2969361
ymax = 42.7209728

centerx = np.array([xmin,xmax]).mean()
centery = np.array([ymin,ymax]).mean()

ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

In [22]:
bathy = loadBathymetry("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/bathymetric_data_BUB17")

wmt = ee.Image('users/dmlmont/TFM/WMt_Bubal')
wmp = ee.Image('users/dmlmont/TFM/WMp_Bubal')

coords_wmp = getLatLonPixels(wmp,ROI)
coords_wmt = getLatLonPixels(wmt,ROI)

wl = getWaterlineCoordinates(wmt,ROI,level = 2)

np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Bat_wl_Bubal.csv", np.vstack((wl,bathy)), delimiter=",")
np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Coords_wmp_Bubal.csv",coords_wmp, delimiter=",")
np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Coords_wmt_Bubal.csv",coords_wmt, delimiter=",")


Creating dictionary:  75%|███████████████████████████████████████████████▎               | 3/4 [00:01<00:00,  2.86it/s]
Creating array | Deleting duplicates:  75%|██████████████████████████████████▌           | 3/4 [00:01<00:00,  2.86it/s]
Creating array | Deleting duplicates: : 6it [00:01,  3.92it/s]                                                         
Done: : 6it [00:01,  5.19it/s]                                

Creating dictionary:  75%|███████████████████████████████████████████████▎               | 3/4 [00:01<00:00,  2.52it/s]
Creating array | Deleting duplicates:  75%|██████████████████████████████████▌           | 3/4 [00:01<00:00,  2.52it/s]
Creating array | Deleting duplicates: : 6it [00:01,  3.45it/s]                                                         
Done: : 6it [00:01,  4.57it/s]                                


In [24]:
xmin = 0.5664387
ymin = 41.9718079
xmax = 0.7096043
ymax = 42.1213370

centerx = np.array([xmin,xmax]).mean()
centery = np.array([ymin,ymax]).mean()

ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

In [25]:
bathy = loadBathymetry("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/bathymetric_data_Canelles_2018")

wmt = ee.Image('users/dmlmont/TFM/WMt_Canelles')
wmp = ee.Image('users/dmlmont/TFM/WMp_Canelles')

coords_wmp = getLatLonPixels(wmp,ROI)
coords_wmt = getLatLonPixels(wmt,ROI)

wl = getWaterlineCoordinates(wmt,ROI)

np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Bat_wl_Canelles.csv", np.vstack((wl,bathy)), delimiter=",")
np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Coords_wmp_Canelles.csv",coords_wmp, delimiter=",")
np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Coords_wmt_Canelles.csv",coords_wmt, delimiter=",")


Creating dictionary:  75%|███████████████████████████████████████████████▎               | 3/4 [00:07<00:02,  2.46s/it]
Creating array | Deleting duplicates:  75%|██████████████████████████████████▌           | 3/4 [00:07<00:02,  2.46s/it]
Creating array | Deleting duplicates: : 6it [00:10,  2.00s/it]                                                         
Done: : 6it [00:10,  1.70s/it]                                

Creating dictionary:  75%|███████████████████████████████████████████████▎               | 3/4 [00:08<00:02,  2.80s/it]
Creating array | Deleting duplicates:  75%|██████████████████████████████████▌           | 3/4 [00:08<00:02,  2.80s/it]
Creating array | Deleting duplicates: : 6it [00:11,  2.24s/it]                                                         
Done: : 6it [00:11,  1.87s/it]                                


In [26]:
xmin = 0.1912200
ymin = 42.1501858
xmax = 0.2537043
ymax = 42.3090388

centerx = np.array([xmin,xmax]).mean()
centery = np.array([ymin,ymax]).mean()

ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

In [27]:
bathy = loadBathymetry("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/bathymetric_data_Grado")

wmt = ee.Image('users/dmlmont/TFM/WMt_Grado')
wmp = ee.Image('users/dmlmont/TFM/WMp_Grado')

coords_wmp = getLatLonPixels(wmp,ROI)
coords_wmt = getLatLonPixels(wmt,ROI)

wl = getWaterlineCoordinates(wmt,ROI)

np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Bat_wl_Grado.csv", np.vstack((wl,bathy)), delimiter=",")
np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Coords_wmp_Grado.csv",coords_wmp, delimiter=",")
np.savetxt("C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/Bat_data/Coords_wmt_Grado.csv",coords_wmt, delimiter=",")


Creating dictionary:  75%|███████████████████████████████████████████████▎               | 3/4 [00:04<00:01,  1.36s/it]
Creating array | Deleting duplicates:  75%|██████████████████████████████████▌           | 3/4 [00:04<00:01,  1.36s/it]
Creating array | Deleting duplicates: : 6it [00:05,  1.08s/it]                                                         
Done: : 6it [00:05,  1.12it/s]                                

Creating dictionary:  75%|███████████████████████████████████████████████▎               | 3/4 [00:04<00:01,  1.52s/it]
Creating array | Deleting duplicates:  75%|██████████████████████████████████▌           | 3/4 [00:04<00:01,  1.52s/it]
Creating array | Deleting duplicates: : 6it [00:05,  1.19s/it]                                                         
Done: : 6it [00:05,  1.03it/s]                                


(6409, 2)